# Model Training

This notebook defines and trains a Keras network to predict, for each order, which of the items in the user's _previous_ order will reappear in it.

This is slightly different from the goal of the Kaggle contest, which is to predict which items from _any_ point in the user's past history will reappear in this order.

Make sure you run the [Data Preparation](1-DataPrep.ipynb) notebook before this one.

Training will be performed on a GPU automatically if your machine has one, or CPU otherwise (assuming you haven't messed with Keras's configuration).

## Problem definition

Conceptually, we're trying to predict a number of items for each order -- a multilabel classification problem.

However, since the candidate items for each order come from a restricted set (the _n_ items in the previous order), this decomposes nicely into _n_ individual binary classification problems. This is generally an easier problem to solve, especially when _n_ is much smaller than the total number of items in the data.

So, each training (or test) instance is actually an item in an order, and the label is 1 if that item also appears in the next order, and 0 otherwise.

This means each order yields _n_ separate training instances, one for each item it contains.

## Network structure

The inputs to the network, for each training instance, are:

* (1) The order that this instance came from
 * This remains the same for all the _n_ instances from this order
* (2) The items in that order which were themselves reorders
 * This also remains the same for all instances from this order
* (3) The user
 * This also remains the same for all instances from this order
* (4) The item itself
 * This is the only input that differs between instances from the same order

The label, for each instance, is a 1 or 0 indicating whether or not it reappears in the user's next order.

All items are represented by an _embedding_, i.e. a vector of floats. These values are learnt during training, along with all the other weights of the network, and items which appear in similar contexts will result in similar embeddings.

A collection of items (i.e. (1) and (2)) is represented by getting the embeddings for those items and simply adding them together, so an empty collection is just a vector of zeros.

Each user is also represented by an embedding, again learnt during training.

The final input to the network, then, is a concatenation of four vectors:

* The order vector (sum of item embeddings in order)
* The reorder vector (sum of reordered item embeddings in order)
* The current item's embedding
* The current user's embedding

Or more visually:

```
|---Order-Vector-6---||--Reorder-Vector-6--||---Item-Vector-13---||---User-Vector-22---|
|---Order-Vector-6---||--Reorder-Vector-6--||---Item-Vector-43---||---User-Vector-22---|
|---Order-Vector-6---||--Reorder-Vector-6--||---Item-Vector-91---||---User-Vector-22---|
...
|---Order-Vector-9---||--Reorder-Vector-9--||---Item-Vector-10---||---User-Vector-71---|
|---Order-Vector-9---||--Reorder-Vector-9--||---Item-Vector-13---||---User-Vector-71---|
...
```

These vectors are fed through two consecutive fully-connected layers, each the same width, and then to a single output node which predicts the label. This is scored against the true label via [cross-entropy loss](https://jamesmccaffrey.wordpress.com/2013/11/05/why-you-should-use-cross-entropy-error-instead-of-classification-error-or-mean-squared-error-for-neural-network-classifier-training/).

## Evaluation metrics

Before training, 5000 randomly-selected order pairs will be set aside as validation data. After each training epoch -- i.e. each pass through the training data -- we'll evaluate the model's predictive power over the validation set, reporting [precision, recall and F1 score](https://en.wikipedia.org/wiki/Precision_and_recall) (balanced F-measure) averaged over these orders.

N.B. This isn't necessarily a perfect predictor of how well the model will do in production, as the training data can potentially contain order pairs that appear chronologically later in the input data than order pairs in the validation data. This is a violation of the "no time machines" rule. But, it's convenient for demonstration purposes. A thorough evaluation would involve a held-out test set consisting of each user's most recent order.

In [1]:
import numpy as np
import keras as k
import sklearn as sk
import tensorflow as tf
import h5py
import os

h5_dir = 'h5/'

Using TensorFlow backend.


## Reopen saved datasets

Pull all the data out of HDF5 file and back into memory. This is much faster than doing random access directly on the files, particularly if you're using a cloud server with network-attached storage.

In [2]:
datafile = h5py.File(os.path.join(h5_dir, 'training_data.h5'), 'r')
orders_dataset = datafile['orders'][:]
reorders_dataset = datafile['reorders'][:]
users_dataset = datafile['users'][:]
items_dataset = datafile['items'][:]
labels_dataset = datafile['labels'][:]
num_rows = datafile['num_rows'][0]
biggest_order_size = datafile['biggest_order_size'][0]
max_product_id = datafile['max_product_id'][0]
max_user_id = datafile['max_user_id'][0]
datafile.close()

## Network helpers

Not all Keras layers support masking (e.g. ignoring product ID 0 which is effectively 'null' in our data).

So we create a new type of layer that removes masked-out values before passing the data on to the next layer. See: https://github.com/fchollet/keras/issues/2728

Also we create a couple of helper functions to let us sum all the vectors coming from a previous layer.

In [3]:
import keras.backend as K
from keras.engine.topology import Layer

class ZeroMaskedEntries(Layer):
    """
    This layer is called after an Embedding layer.
    It zeros out all of the masked-out embeddings.
    It also swallows the mask without passing it on.
    You can change this to default pass-on behavior as follows:

    def compute_mask(self, x, mask=None):
        if not self.mask_zero:
            return None
        else:
            return K.not_equal(x, 0)
    """

    def __init__(self, **kwargs):
        self.support_mask = True
        super(ZeroMaskedEntries, self).__init__(**kwargs)

    def build(self, input_shape):
        self.output_dim = input_shape[1]
        self.repeat_dim = input_shape[2]

    def call(self, x, mask=None):
        mask = K.cast(mask, 'float32')
        mask = K.repeat(mask, self.repeat_dim)
        mask = K.permute_dimensions(mask, (0, 2, 1))
        return x * mask

    def compute_mask(self, input_shape, input_mask=None):
        return None

def sum_layer_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3 
    return (shape[0], shape[2])

def sum_layer(x):
  return K.sum(x, axis=1, keepdims=False)

## Network structure

The following code defines the structure of the neural network, as described in the notes at the top of this file.

The constants at the top define the dimensionality of the user and product embeddings, and the hidden layer sizes. The final dimensionality of the concatenated input vector is:

    (product_embedding_size * 3) + user_embedding_size

because we have order, reorder and current item vectors, again as described above.

The hidden layers are [exponential linear units](https://arxiv.org/abs/1511.07289) with [dropout](http://jmlr.org/papers/v15/srivastava14a.html). The network is trained using the [ADAM optimizer](https://arxiv.org/abs/1412.6980).

### Note

When you run the following cell, no data actually gets processed.

The Keras API calls here just _define_ the network structure -- think of it like a declarative DSL for describing a network.

We'll actually pass data into the network, and evaluate its output, later on.

In [13]:
# Layer size constants
user_embedding_size = 50
product_embedding_size = 50
hidden_1_size = 500
hidden_2_size = 500
hidden_3_size = 250

# Activation function for the hidden layers
activation = 'elu'

# Dropout rate for the hidden layers
dropout = 0.33

# Initial learning rate
learning_rate = 0.01

# Input layers for the four datasets

user_input = k.layers.Input(shape=(1,), name='user_input')

order_input = k.layers.Input(shape=(biggest_order_size,), name='order_input')

reorder_input = k.layers.Input(shape=(biggest_order_size,), name='reorder_input')

item_input = k.layers.Input(shape=(1,), name='item_input')

# Set up the product embeddings -- this gets reused to map the order, reorder and
# individual item IDs into embeddings
# Think of it as a lookup table mapping IDs to vectors of floats

product_embedding = k.layers.Embedding(
  input_dim=max_product_id + 1, output_dim=product_embedding_size,
  mask_zero=True, name='product_embedding')

# Set up a separate user embedding table

user_embedding = k.layers.Embedding(
  input_dim=max_user_id + 1, output_dim=user_embedding_size, name='user_embedding')

# Sum up the embeddings in the order and reorder sets into order and reorder 

order_embedding = ZeroMaskedEntries()(product_embedding(order_input))

order_vector = k.layers.Lambda(sum_layer, sum_layer_output_shape, name='order_vector')(
  order_embedding)

reorder_embedding = ZeroMaskedEntries()(product_embedding(reorder_input))

reorder_vector = k.layers.Lambda(sum_layer, sum_layer_output_shape, name='reorder_vector')(
  reorder_embedding)

# Flatten the single item embedding into a vector
# Hack: Flatten also doesn't support masks, but we don't need to do anything special
# The item embeddings will never be masked out, there's always exactly one in each

flattened_item = k.layers.Lambda(
  lambda x: x, output_shape=lambda s:s, name='flattened_item')

item_vector = k.layers.Flatten(name='item_vector')(
  flattened_item(product_embedding(item_input)))

# Flatten the single user embedding into a vector

user_vector = k.layers.Flatten(name='user_vector')(
  user_embedding(user_input))

full_input_vector = k.layers.concatenate(
  [order_vector, reorder_vector, item_vector, user_vector], name="full_input_vector")

# Define the hidden layers

hidden_1 = k.layers.Dense(hidden_1_size, activation=activation, name='hidden_1')(full_input_vector)
dropout_1 = k.layers.Dropout(dropout, name='dropout_1')(hidden_1)

hidden_2 = k.layers.Dense(hidden_2_size, activation=activation, name='hidden_2')(dropout_1)
dropout_2 = k.layers.Dropout(dropout, name='dropout_2')(hidden_2)

hidden_3 = k.layers.Dense(hidden_3_size, activation=activation, name='hidden_3')(dropout_2)
dropout_3 = k.layers.Dropout(dropout, name='dropout_3')(hidden_3)

output = k.layers.Dense(1, activation='sigmoid', name='output')(hidden_3)

# Compile the model

model = k.models.Model(
  inputs=[order_input, reorder_input, user_input, item_input], outputs=output)

model.compile(
  optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate), loss='binary_crossentropy')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
order_input (InputLayer)         (None, 145)           0                                            
____________________________________________________________________________________________________
reorder_input (InputLayer)       (None, 145)           0                                            
____________________________________________________________________________________________________
item_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
product_embedding (Embedding)    multiple              2484450     order_input[0][0]                
                                                                   reorder_input[0][0]     

### TODO model graph

## Training

The following sections set up the training process.

We read the data in batches of 1000 order pairs at a time, although each of these gets expanded into one training instance per item in the order, as described earlier.

The data is shuffled before training starts, and again at the end of each training epoch, because data with non-random ordering can cause batch-based optimization algorithms to behave in unexpected ways and perform badly.

Note that shuffling the data is done indirectly, via a numeric index into the input datasets, rather than by trying to shuffle them in-place but in a synchronized manner. This makes the code much simpler, and is faster too.

We set aside 5000 order pairs as the validation set.

In [5]:
batch_size = 1000
num_valid = 5000

index = np.arange(num_rows, dtype=np.uint32)
np.random.shuffle(index)
index_valid = index[:num_valid]
index_train = index[num_valid:]

Next we define a couple of helper functions.

`make_training_data` is called once per batch, on a list of indices into the training data. It retrieves the records corresponding to those indices from the underlying datasets, performs some minor adjustments and returns them as Numpy arrays.

`validate` is called once per epoch, i.e. after one whole run through the training data. It calculates some metrics on the validation data and returns them.

In [10]:
def make_training_data(indices):
  
  # Get labels for this batch's instances, and stack them into a single vector
  labels_separate = labels_dataset[indices]
  labels = np.hstack(labels_separate)
  
  # Get the arrays of ordered item IDs and reordered item IDs for each order
  # (these are already zero-padded to the same length)
  orders_separate = orders_dataset[indices]
  reorders_separate = reorders_dataset[indices]
  
  # From the length of each individual label array, we know how many items
  # are in each of the orders
  order_lengths = [len(items) for items in labels_separate]

  # Repeat each order array and each reorder array that many times
  orders_data = np.repeat(orders_separate, order_lengths, axis=0)
  reorders_data = np.repeat(reorders_separate, order_lengths, axis=0)
  
  # Also repeat each user ID the appropriate number of times
  user_ids = np.repeat(users_dataset[indices], order_lengths)
  
  # We don't need to repeat the item IDs, each training instance corresponds
  # to a single item, to we just concatenate them into a single array
  item_ids = np.hstack(items_dataset[indices])
  
  # Now we return a bunch of things:
  #   orders_data is a zero-padded matrix of IDs, num instances x biggest_order_size
  #   reorders_data is a zero-padded matrix of IDs, same size as orders_data
  #   user_ids is a vector of IDs, length = num instances
  #   item_ids is a vector of IDs, length = num instances
  #   labels is a vector of 0/1 values, length = num instances
  #   order_lengths is a vector of ints, length = batch size
  # where "batch size" is the number of orders in the batch, and
  # "num instances" is the total number of items in the batch
  # (the sum of all order sizes in the batch)
  return (orders_data, reorders_data, user_ids, item_ids, labels, order_lengths)

def validate(model):
  
  # Feed the validation data into the model, with labels, and get the
  # value of the loss function defined by the model (i.e. cross-entropy)
  loss = model.test_on_batch({'order_input': valid_order_data,
                              'reorder_input': valid_reorder_data,
                              'item_input': valid_item_ids,
                              'user_input': valid_user_ids}, valid_labels)
  
  # Feed the validation data into the model WITHOUT labels, and get
  # the output of the final layer of the model
  output = model.predict_on_batch({'order_input': valid_order_data,
                                   'reorder_input': valid_reorder_data,
                                   'item_input': valid_item_ids,
                                   'user_input': valid_user_ids})
  
  preds = np.where(output > 0.5, 1, 0)
  preds_split = np.split(preds, valid_split_intervals)[:-1]
  precision = np.zeros(num_valid)
  recall = np.zeros(num_valid)
  for i, (preds, labels) in enumerate(zip(preds_split, valid_labels_split)):
    matches = float(sum(preds.T[0] * labels))
    if sum(preds) > 0:
      precision[i] = matches / sum(preds)
    if sum(labels) > 0:
      recall[i] = matches / sum(labels)
  f1 = np.nan_to_num((2 * precision * recall) / (precision + recall))
  return (loss, precision.mean(), recall.mean(), f1.mean())

In [7]:
valid_order_data, valid_reorder_data, valid_user_ids, valid_item_ids, valid_labels, valid_lengths = make_training_data(index_valid)
valid_split_intervals = np.cumsum(valid_lengths)
valid_labels_split = np.split(valid_labels, valid_split_intervals)[:-1]

In [11]:
steps_per_epoch = num_rows // batch_size
epochs = 1

print("Batch size: %d" % batch_size)
print("Training examples: %d" % num_rows)
print("Batches per epoch: %d" % steps_per_epoch)

def train_data_generator():
  for chunk in np.array_split(index_train, steps_per_epoch):
    orders_data, reorders_data, user_ids, item_ids, labels, item_lengths = make_training_data(chunk)
    yield ({'order_input': orders_data,
            'reorder_input': reorders_data,
            'item_input': item_ids,
            'user_input': user_ids},
           {'output': labels})
  return

# TODO early stopping, checkpointing, learning rate reduction

for epoch in range(epochs):
  
  print("Starting epoch %d" % epoch)
  
  labels_sample = np.hstack(labels_dataset[np.sort(index_train[0:num_valid]).tolist()])
  class_weights = sk.utils.class_weight.compute_class_weight('balanced', [0, 1], labels_sample)
  del labels_sample
  
  model.fit_generator(train_data_generator(), steps_per_epoch=steps_per_epoch, epochs=1, max_q_size=1,
                      class_weight={0: class_weights[0], 1: class_weights[1]})
  
  print("Validation: loss = %0.5f, P = %0.5f, R = %0.5f, F = %0.5f" % validate(model))
  print("Shuffling training data")
  np.random.shuffle(index_train)

Batch size: 1000
Training examples: 2933665
Batches per epoch: 2933
Starting epoch 0
Epoch 1/1
2932/2933 [============================>.] - ETA: 0s - loss: 0.6191

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/andrew/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/andrew/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/andrew/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
StopIteration



2933/2933 [==============================] - 1402s - loss: 0.6191  
Validation: loss = 0.60755, P = 0.34115, R = 0.48297, F = 0.36742
Shuffling training data


/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in divide


50/50, 500, 500, elu/0.33, adam 0.01:

0: loss = 0.60755, P = 0.34115, R = 0.48297, F = 0.36742

50/50, 200, 200, elu/0.33, adam 0.01:

0: loss = 0.57991, P = 0.34420, R = 0.44656, F = 0.35717

50/50, 200, 200, relu/0.33, adam 0.01:

0: loss = 0.58145, P = 0.33895, R = 0.44538, F = 0.35334